# Transfer Learning with TensorFlow Part 3: Scaling Up (Food Vision Mini)

We've seen the power of transfer learning feature extraction and fine-tuning, now it's time to scale up to all of the classes in Food101 (101 classes of food).

Our goal is to beat the original Food101 paper with 10% of the training data (leveraging the power of deep learning)